<a href="https://colab.research.google.com/github/janithcyapa/SmartHVAC/blob/main/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!pip install psychrolib

In [77]:
import psychrolib as psy
import numpy as np
import math

In [ ]:
# Use SI units to standard
psy.SetUnitSystem(psy.SI)

In [82]:
# Control Signal
Control_Air_AF = 0.5 # 0 ~ 1
Control_Vent_AF = 0.1 # 0 ~ 1
Control_Cooling_PW = 1 # 0 ~ 1
Control_Heeting_PW = 0.3 # 0 ~ 1

# Room COnfig
dt = 60 # s Time to simulate
N_Occ = 3 # Number of Occupants in the room
Q_Equ = 1000 # W - Sensible Heat from Equipment/Lights
T_room = 25 # °C - Initial dry bulb temperature
RH_room = 0.5 # 0 ~ 1 - Initial reletive humidity
P_air = 101325        # Pa -  Atmospheric pressure (sea level)
L_room = 5 # m - Length of the room
W_room = 4 # m - Width of the room
H_room = 2.5 # m - Height of the room
A_window = 4 # m^2 - Area of the window
A_wall = (L_room + W_room) * H_room - A_window # m^2 - Area of the wall
V_room = L_room * W_room * H_room # m^3 - Volume of the room
# Room Conditions
# Humidity ratio
W_room = psy.GetHumRatioFromRelHum(T_room, RH_room, P_air) # kg water/kg dry air
# Enthalpy
h_room = psy.GetMoistAirEnthalpy(T_room, W_room ) # J/kg
h_fg_room = 2441000 # J/kg
# Air dencity
p_air = psy.GetMoistAirDensity(T_room, W_room, P_air)
# Specific heat of moist air
Cp_Air = 1005 # J/kg-K

# Weather Data
T_out = 35 # °C - Outside temperature
RH_out = 0.7 # 0 ~ 1 - Outside reletive humidity
W_out = psy.GetHumRatioFromRelHum(T_out, RH_out, P_air) # kg water/kg dry air

# AHU Config
Max_Air_FR = 0.1 # m^3/s Maximum airflow rate at maximum setting
P_fan = 5.0  # W Fan power
Max_Vent_FR = 0.1 # m^3/s Maximum ventilation airflow rate at maximum setting
Max_Cooling_PW = 1500 # W Maximum Cooling load of cooling pump
Max_Heating_PW = 1000 # W Maximum heating load of cooling pump

# Calculated Parameters
# Assume Air Flow, Cooling power and Heating power varies linearly with control input
Air_AF = Max_Air_FR * Control_Air_AF # m^3/s
Vent_AF = Max_Air_FR * Control_Vent_AF # m^3/s
Cooling_PW = Max_Cooling_PW * Control_Cooling_PW # W
Heating_PW = Max_Heating_PW * Control_Heeting_PW # W

# Standard Definitions
# 1.0 Btu/h = 0.293 W
# Sensible Heat from Occupants
Q_S_Per_Occ = 200 * 0.293 # (W) - Typically 200 Btu/h per person for light activity
#Latent Heat from Occupants
Q_L_Per_Occ = 180 * 0.293 # (W) - Typically 180 Btu/h per person for light activity
# Other latent heat sources
Q_L_Other = 0 # (W) - Assume no latent heat for now


U_wall = 0.284 # W/m^2.K
U_window = 1.987 # W/m^2.K

print("Parameters")
print(f"Q_S_Per_Occ: {Q_S_Per_Occ} W")
print(f"Q_L_Per_Occ: {Q_L_Per_Occ} W")
print(f"W_room: {W_room} kg water/kg dry air")
print(f"h_room: {h_room} J/kg")
print(f"h_fg_room: {h_fg_room} J/kg")
print(f"p_air: {p_air} kg/m^3")
print(f"Cp_Air: {Cp_Air} kJ/kg-K")
print("")

# Total Room Sensible and Latent Heat Loads
Q_S_envelope = U_wall * A_wall  * (T_out - T_room) + U_window * A_window  * (T_out - T_room) # (W)
Q_S_total = Q_S_Per_Occ * N_Occ + Q_Equ + Q_S_envelope # (W) - add Q_solar_sensible if applicable
Q_L_total = Q_L_Per_Occ * N_Occ + Q_L_Other # (W)
print("Total Room Sensible and Latent Heat Loads")
print(f"Q_S_envelope: {Q_S_envelope} W")
print(f"Q_S_total: {Q_S_total} W")
print(f"Q_L_total: {Q_L_total} W")
print("")

print("Room air")
print(f"T_room: {T_room} °C")
print(f"W_room: {W_room} kg water/kg dry air")
print(f"RH_room: {RH_room}")
print(f"hg_room: {h_room} J/kg")

print("")

# # Determine the Required Supply Air Conditions
# T_SA_required = T_room - (Q_S_total / (p_air * Cp_Air * Air_AF))
# W_SA_required = W_room - (Q_L_total / (p_air * h_fg_room* Air_AF))
# RH_SA_required = psy.GetRelHumFromHumRatio(T_SA_required, max(W_SA_required, 0), P_air)
# hg_SA_required = psy.GetMoistAirEnthalpy(T_SA_required, max(W_SA_required, 0) ) # J/kg
# print("Determine the Required Supply Air Conditions")
# print(f"T_SA_required: {T_SA_required} °C")
# print(f"W_SA_required: {W_SA_required} kg water/kg dry air")
# print(f"RH_SA_required: {RH_SA_required}")
# print(f"hg_SA_required: {hg_SA_required}")
# print("")

# Mixed Air Dry-bulb Temperature (T_MA)
T_MA = (Vent_AF * T_out + (Air_AF - Vent_AF) * T_room) / Air_AF
W_MA = (Vent_AF * W_out + (Air_AF - Vent_AF) * W_room) / Air_AF
RH_MA = psy.GetRelHumFromHumRatio(T_MA, max(W_MA, 0), P_air)
h_MA = psy.GetMoistAirEnthalpy(T_MA, max(W_MA, 0) ) # J/kg
print("Mixed Air Dry-bulb Temperature (T_MA)")
print(f"T_MA: {T_MA} °C")
print(f"W_MA: {W_MA} kg water/kg dry air")
print(f"RH_MA: {RH_MA}")
print(f"h_MA: {h_MA} J/kg")
print("")

# Simulate Cooling Coil Process and Determine Air Condition After Cooling Coil
h_Coil = h_MA - (Cooling_PW / (p_air * Air_AF))
# Assume Near-Saturation or Saturation
RH_Coil = 0.9

# Iterate over a range of temperatures to find T where h matches h_coil at RH = 0.9
T_min, T_max = 5.0, 20.0  # Reasonable range for coil leaving temperature (°C)
T_step = 0.01  # Small step for precision
tolerance = 100  # Enthalpy tolerance in J/kg

T_Coil = None
W_Coil = None
h_Calc = None

for T in np.arange(T_min, T_max, T_step):
    W = psy.GetHumRatioFromRelHum(T, RH_Coil, P_air)
    h = psy.GetMoistAirEnthalpy(T, W)
    if abs(h - h_Coil) < tolerance:
        T_Coil = T
        W_Coil = W
        h_calc = h
        break
RH_Coil = psy.GetRelHumFromHumRatio(T_Coil, W_Coil, P_air)
T_dew = psy.GetTDewPointFromRelHum(T_Coil, RH_Coil)

print("Air Condition After Cooling Coil")
print(f"T_Coil: {T_Coil} °C")
print(f"W_Coil: {W_Coil} kg water/kg dry air")
print(f"RH_Coil: {RH_Coil}")
print(f"h_Coil: {h_Coil} J/kg")
print(f"T_dew: {T_dew} °C")
print("")

# Fan Heat Addition (for Draw-Through AHU)
ΔT_fan = (P_fan) / (p_air * Cp_Air * Air_AF)
T_fan  = T_Coil + ΔT_fan
W_fan = W_Coil
RH_fan = psy.GetRelHumFromHumRatio(T_fan, W_fan, P_air)
h_fan = psy.GetMoistAirEnthalpy(T_fan, W_fan)
print("Fan Heat Addition (for Draw-Through AHU)")
print(f"ΔT_fan: {ΔT_fan} °C")
print("")
print(f"T_fan: {T_fan} °C")
print(f"W_fan: {W_fan} kg water/kg dry air")
print(f"RH_fan: {RH_fan}")
print(f"h_fan: {h_fan} J/kg")
print("")

#  Simulate Heating Coil Process and Determine Final Supply Air Conditions
T_Out = T_fan + (Heating_PW / (p_air * Cp_Air * Air_AF))
W_Out = W_fan
RH_Out = psy.GetRelHumFromHumRatio(T_Out, max(W_Out, 0), P_air)
h_Out = psy.GetMoistAirEnthalpy(T_Out, max(W_Out, 0) ) # J/kg
print("Simulate Heating Coil Process and Determine Final Supply Air Conditions")
print(f"T_Out: {T_Out} °C")
print(f"W_Out: {W_Out} kg water/kg dry air")
print(f"RH_Out: {RH_Out}")
print(f"h_Out: {h_Out} J/kg")
print("")

# The room is well-mixed, so the air inside has uniform temperature and humidity.
# The air exchange is continuous, with equal inlet and outlet flow rates ($   Q   $).
# No heat gains/losses from walls, occupants, or equipment (unless specified).
# No moisture sources/sinks (e.g., no evaporation or condensation) inside the room.
# Air properties (specific heat, density) are approximately constant over the temperature range.
# The process is at standard atmospheric pressure (101.325 kPa) unless otherwise specified.
# dT/dt = Q/V (Tin- Troom)

T_final = T_Out +(T_room-T_Out)* math.exp(-(Air_AF / V_room) * dt)
W_final = W_Out +(W_room-W_Out)* math.exp(-(Air_AF / V_room) * dt)
RH_final = psy.GetRelHumFromHumRatio(T_final, max(W_final, 0), P_air)
h_final = psy.GetMoistAirEnthalpy(T_final, max(W_final, 0) ) # J/kg
print("Simulate Final Air Temp")
print(f"T_final: {T_final} °C")
print(f"W_final: {W_final} kg water/kg dry air")
print(f"RH_final: {RH_final}")
print(f"h_final: {h_final} J/kg")
print("")

Parameters
Q_S_Per_Occ: 58.599999999999994 W
Q_L_Per_Occ: 52.739999999999995 W
W_room: 0.009881043690749623 kg water/kg dry air
h_room: 50321.958802184665 J/kg
h_fg_room: 2441000 J/kg
p_air: 1.17695818641638 kg/m^3
Cp_Air: 1005 kJ/kg-K

Total Room Sensible and Latent Heat Loads
Q_S_envelope: 132.01999999999998 W
Q_S_total: 1307.82 W
Q_L_total: 158.21999999999997 W

Room air
T_room: 25 °C
W_room: 0.009881043690749623 kg water/kg dry air
RH_room: 0.5
hg_room: 50321.958802184665 J/kg

Mixed Air Dry-bulb Temperature (T_MA)
T_MA: 27.0 °C
W_MA: 0.012936662590369461 kg water/kg dry air
RH_MA: 0.5787686569624064
h_MA: 60166.27233380238 J/kg

Air Condition After Cooling Coil
T_Coil: 13.139999999999826 °C
W_Coil: 0.00846410447132231 kg water/kg dry air
RH_Coil: 0.9000000000000002
h_Coil: 34676.83664590896 J/kg
T_dew: 11.537979789175939 °C

Fan Heat Addition (for Draw-Through AHU)
ΔT_fan: 0.0845420752500611 °C

T_fan: 13.224542075249888 °C
W_fan: 0.00846410447132231 kg water/kg dry air
RH_fan: 0.